In [1]:
%pip install --upgrade google-cloud-aiplatform==1.35.0 google-cloud-documentai==2.20.1 backoff==2.2.1 --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.4/298.4 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 76.1 MB/s eta 0:00:00:00:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel

In [2]:
import sys

if "google.colab" in sys.modules:
    # Automatically restart kernel after installs so that your environment can access the new packages
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
else:
    # Otherwise, attempt to discover local credentials as described on https://cloud.google.com/docs/authentication/application-default-credentials
    pass

In [2]:
# %pip install backoff
%pip install tenacity
%pip install pandas
%pip install vertexai

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 109.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 56.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Enable APIs for the project

In [1]:
from __future__ import annotations
import backoff
from tenacity import retry, stop_after_attempt, wait_random_exponential
from google.api_core.exceptions import ResourceExhausted
from google.api_core.client_options import ClientOptions
from google.api_core.exceptions import AlreadyExists
from google.cloud import documentai
import numpy as np
import glob
import os
from typing import Dict, List
import pandas as pd
from logging import error
import re
import textwrap
from typing import Tuple, List
import vertexai
from vertexai.language_models import TextEmbeddingModel, TextGenerationModel

In [2]:
PROJECT_ID = !gcloud config get project
PROJECT_ID = PROJECT_ID.n
LOCATION = "europe-west2"
LOCATION_DEPLOY = "europe-west2" #Location to deploy GCP resources

!gcloud services enable documentai.googleapis.com storage.googleapis.com aiplatform.googleapis.com

ERROR: (gcloud.services.enable) User [playpen-416552-consumer-sa@playpen-416552.iam.gserviceaccount.com] does not have permission to access projects instance [playpen-416552] (or it may not exist): Permission denied to enable service [documentai.googleapis.com]
Permission denied to enable service [storage.googleapis.com]
Permission denied to enable service [aiplatform.googleapis.com]
Help Token: ARD_zUZct_Km9_bDTlevjRPDWxevW2n31xsZDpLsKE4f6ZjDRQhCyv_Tpu456SqM44E3_B-BbNuX6FmfxIDoRvCFSx1AFESepWA9ydBRR6hhIoaW
- '@type': type.googleapis.com/google.rpc.PreconditionFailure
  violations:
  - subject: ?error_code=110002&service=serviceusage.googleapis.com&permission=serviceusage.services.enable&resource=playpen-416552
    type: googleapis.com
- '@type': type.googleapis.com/google.rpc.ErrorInfo
  domain: serviceusage.googleapis.com
  metadata:
    permission: serviceusage.services.enable
    resource: playpen-416552
    service: serviceusage.googleapis.com
  reason: AUTH_PERMISSION_DENIED


In [8]:
# TODO(developer): Edit these variables before running the code.
project_id = PROJECT_ID

# See https://cloud.google.com/document-ai/docs/regions for all options.
location = LOCATION

# Must be unique per project, e.g.: "My Processor"
processor_display_name = "YOUR_PROCESSOR_DISPLAY_NAME"

# You must set the `api_endpoint` if you use a location other than "us".
client_options = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")


def create_processor(
    project_id: str, location: str, processor_display_name: str
) -> documentai.Processor:
    client = documentai.DocumentProcessorServiceClient(client_options=client_options)

    # The full resource name of the location
    # e.g.: projects/project_id/locations/location
    parent = client.common_location_path(project_id, location)

    # Create a processor
    return client.create_processor(
        parent=parent,
        processor=documentai.Processor(
            display_name=processor_display_name, type_="OCR_PROCESSOR"
        ),
    )


try:
    processor = create_processor(project_id, location, processor_display_name)
    print(f"Created Processor {processor.name}")
except AlreadyExists as e:
    print(
        f"Processor already exits, change the processor name and rerun this code. {e.message}"
    )

PermissionDenied: 403 Cloud Document AI API has not been used in project playpen-416552 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/documentai.googleapis.com/overview?project=playpen-416552 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry. [links {
  description: "Google developers console API activation"
  url: "https://console.developers.google.com/apis/api/documentai.googleapis.com/overview?project=playpen-416552"
}
, reason: "SERVICE_DISABLED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "documentai.googleapis.com"
}
metadata {
  key: "consumer"
  value: "projects/playpen-416552"
}
]

In [4]:
from google.cloud import aiplatform

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [5]:
# Extract text from PDF

import pdfplumber
import pandas as pd

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        pdf_text=""
        for page in pdf.pages:
            pdf_text += page.extract_text()
    return pdf_text
        

In [7]:
text_pdf = extract_text_from_pdf("bp-sustainability-report-2022.pdf")

In [9]:
#split into sentences
def text_to_sentences(text):
    sentences = text.split('\n')
    return sentences

def create_sentence_chunks(sentences, chunk_size):
    sentence_chunks = [sentences[i:i+chunk_size] for i in range(0, len(sentences), chunk_size)]
    return sentence_chunks

def text_to_paragraph(text, chunk_size):
    return create_sentence_chunks(text_to_sentences(text), chunk_size)
    
chunk_size = 15
sentence_chunks = text_to_paragraph(text_pdf, chunk_size)
sentence_chunks

[['Reimagining energy',
  'for people and our planet',
  'bp sustainability report 2022Contents',
  'Sustainability at bp Getting to net zero 16 Engaging stakeholders 52 About our 2022 reporting',
  'Our sustainability frame 1 Our net zero ambition and aims 17 Engaging with our stakeholders 53',
  'Since we launched our sustainability frame we',
  'Our strategy and performance 2 Getting to net zero introduction 18 have worked to further embed sustainability',
  'Our reporting 56',
  'Letter from our chief executive officer 3 Net zero aims 1-5 update 19 across bp. We have also made progress',
  'Materiality 57 in a number of areas, which are covered in',
  'Letter from our EVP, strategy, Consistency of our ambition and',
  'About our reporting 58 this report.',
  'sustainability & ventures 4 aims with the Paris goals 20',
  '2022 at a glance 5 Five aims to get bp to net zero 21 Glossary 59 Our corporate reporting suite for 2022',
  'Assurance statement 61 comprises several elements, inc

In [13]:
def paragraphs_to_df(paragraphs):
    data = []

    # Iterate through sentence chunks
    for idx, chunk in enumerate(sentence_chunks, start=1):
        # Merge sentences within the chunk into one string
        merged_text = ''.join(chunk)
        # Append data to the list
        data.append({
            'paragraph_number':idx,
            'text':merged_text
        })
    # DataFrame with different paragraphs
    return pd.DataFrame(data)
    

In [14]:
df = paragraphs_to_df(sentence_chunks)

In [16]:
# Call the GCP models
generation_model = TextGenerationModel.from_pretrained("text-bison@001")
embedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")


# # This decorator is used to handle exceptions and apply exponential backoff in case of ResourceExhausted errors.
# # It means the function will be retried with increasing time intervals in case of this specific exception.
# @retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(5))
# def text_generation_model_with_backoff(**kwargs):
#     return generation_model.predict(**kwargs).text


# @retry(wait=wait_random_exponential(min=10, max=120), stop=stop_after_attempt(5))
# def embedding_model_with_backoff(text=[]):
#     embeddings = embedding_model.get_embeddings(text)
#     return [each.values for each in embeddings][0]

In [17]:
# # Adding the embeddings (vector/number representation) of each chunk as a separate column
# df["embedding"] = df["text"].apply(lambda x: embedding_model_with_backoff([x]))
# df["embedding"] = df.embedding.apply(np.array)
# df.head(2)


RetryError: RetryError[<Future at 0x7f197b431ad0 state=finished raised ResourceExhausted>]

In [20]:
import json
import time

import numpy as np
import pandas as pd

def get_embedding(text):
    get_embedding.counter += 1
    try:
        if get_embedding.counter % 100 == 0:
            time.sleep(3)
        return embedding_model.get_embeddings([text])[0].values #Send request to embedding model
    except:
        print('waiting for 60 secs')
        time.sleep(60)
        return embedding_model.get_embeddings([text])[0].values #Send request to embedding model


get_embedding.counter = 0

# This may take several minutes to complete.
df["embedding"] = df["text"].apply(lambda x: get_embedding(x))

waiting for 60 secs


In [35]:
df.head()

,paragraph_number,text,embedding,dot_product
0,1,Reimagining energyfor people and our planetbp ...,"[-0.04452475160360336, -0.025542210787534714, ...",0.672430
1,2,2022 sustainability highlights 6 Aim 1: Net ze...,"[-0.0057776751928031445, -0.028894774615764618...",0.699629
2,3,Policies and practices 12 bp Annual Report 202...,"[-0.01899087429046631, -0.025088345631957054, ...",0.657536
3,4,Five aims to care for the planet 46 Deloitte h...,"[-0.011461344547569752, -0.03942526504397392, ...",0.662655
4,5,lenses: our strong foundations built over many...,"[-0.04292025789618492, -0.033842723816633224, ...",0.635116


In [38]:
def get_context_from_question(
    question: str, vector_store: pd.DataFrame, sort_index_value: int = 2
) -> Tuple[str, pd.DataFrame]:
    query_vector = np.array(get_embedding(question))
    vector_store["dot_product"] = vector_store["embedding"].apply(
        lambda row: np.dot(row, query_vector)
    )
    # Similarity matching by dot product 
    top_matched = vector_store.sort_values(by="dot_product", ascending=False)[
        :sort_index_value
    ].index
    top_matched_df = vector_store.loc[top_matched, ["paragraph_number", "text"]]
    context = "\n".join(top_matched_df["text"].values)
    return context, top_matched_df

In [53]:
%%time
# your question for the documents
question = "Give me the scope 1 emissions in 2021?"

# get the custom relevant chunks from all the chunks in vector store.
context, top_matched_df = get_context_from_question(
    question,
    vector_store=df,
    sort_index_value=2,  # Top N results to pick from embedding vector search
)
# top 5 data that has been picked by model based on user question. This becomes the context.
top_matched_df

CPU times: user 19 ms, sys: 0 ns, total: 19 ms
Wall time: 182 ms


,paragraph_number,text
87,88,"‘Processing’, ‘Transportation’ or Distribution..."
70,71,"projects, including flaring and venting reduct..."


In [56]:
df.iloc[70]['text'] + df.iloc[87]['text']

'projects, including flaring and venting reduction,energy efficiency, electrification and CCS.Progress and targets45.5Reduction in emissions against the 2019 baseline Scope 1 (direct) and Scope 2 (indirect) GHG(absolute basis). emissions (operational control boundary)b35.6 (MtCOe)16% 2020 performance 231.9 60 5.235% 2021 performance3.841% 2022 performance 49.240 2.441.7 1.520% 2025 target‘Processing’, ‘Transportation’ or DistributionBioproducts 47 44 43 43 Scope 1, 2 or 3 emissions:emissions in the graphic above).Power productsc 57 59 56 52 y Such as when someone drives a vehicleOur progress in 2022 So, while aim 3 considers the emissions as relatingusing fuel we sold, or heats their home withto the ‘lifecycle’ of the product, the GHG emissionsIn 2022 the average carbon intensity of sold energy Share of energy delivered per energy natural gas we sold. Aim 3 covers those bp reports under our aim 3 can be thought of asproducts\uf0ab was 77gCOe/MJ. This represents a 2% product typebd emis

In [55]:
# Prompt for Q&A which takes the custom context found in last step.
prompt = f""" Answer the question as precise as possible using the provided context. \n\n
            Context: \n {context}?\n
            Question: \n {question} \n
            Answer:
          """

# Call the PaLM API on the prompt.
print("PaLM Predicted:", text_generation_model_with_backoff(prompt=prompt), "\n\n")

PaLM Predicted: 3.8 MtCOe 


